# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846873773526                   -0.70    4.8         
  2   -7.852321703699       -2.26       -1.53    1.0   17.1ms
  3   -7.852617664064       -3.53       -2.56    1.5   19.1ms
  4   -7.852646024980       -4.55       -2.91    2.5   22.7ms
  5   -7.852646538081       -6.29       -3.24    1.2   18.2ms
  6   -7.852646679983       -6.85       -4.16    1.0   17.4ms
  7   -7.852646686693       -8.17       -5.23    2.2   76.2ms
  8   -7.852646686727      -10.46       -5.64    1.5   19.6ms
  9   -7.852646686729      -11.93       -5.70    1.5   19.5ms
 10   -7.852646686730      -11.89       -6.94    1.0   17.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846893638313                   -0.70           4.5         
  2   -7.852552471501       -2.25       -1.63   0.80    2.0    204ms
  3   -7.852638470235       -4.07       -2.73   0.80    1.0   16.0ms
  4   -7.852646449418       -5.10       -3.32   0.80    2.0   19.8ms
  5   -7.852646672663       -6.65       -4.15   0.80    1.2   17.2ms
  6   -7.852646686459       -7.86       -4.67   0.80    1.8   19.2ms
  7   -7.852646686714       -9.59       -5.57   0.80    1.2   17.3ms
  8   -7.852646686730      -10.81       -6.53   0.80    2.0   20.4ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.451749e+01     4.240984e+00
 * time: 0.047482967376708984
     1     1.078060e+00     2.221504e+00
 * time: 0.2195589542388916
     2    -1.598047e+00     2.657162e+00
 * time: 0.23782801628112793
     3    -3.468775e+00     2.199733e+00
 * time: 0.2639799118041992
     4    -4.778464e+00     1.712739e+00
 * time: 0.2900409698486328
     5    -6.409892e+00     1.199883e+00
 * time: 0.31603193283081055
     6    -7.188167e+00     6.144368e-01
 * time: 0.34215712547302246
     7    -7.510976e+00     4.528192e-01
 * time: 0.3603990077972412
     8    -7.647902e+00     1.312943e-01
 * time: 0.37853097915649414
     9    -7.713704e+00     1.787357e-01
 * time: 0.3965480327606201
    10    -7.774872e+00     1.007326e-01
 * time: 0.4145359992980957
    11    -7.815696e+00     6.224306e-02
 * time: 0.43245410919189453
    12    -7.838653e+00     6.304134e-02
 * time: 0.450451135635376
    13    -7.846711e+00     4.740414e-02
 * time: 0.4687

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846888791783                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645915497                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.51s
  3   -7.852646686730      -13.28       -7.25    127ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.8776515860180296e-7
|ρ_newton - ρ_scfv| = 1.273967503004968e-7
|ρ_newton - ρ_dm|   = 6.172851761312806e-10
